In [2]:
!pip install tenseal
!pip install numpy
!pip install matplotlib

In [3]:
import copy
import tenseal as ts
from time import time
import numpy as np
import math
#from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [4]:
def max_with_lp(x1, x2, x3, x4, p=2):
  #alpha = ts.plain_tensor(1/p)
  alpha = 1/p
  return taylor_series_sqrt_x_plus_1_alpha((x1**p + x2**p+ x3**p+ x4**p) - 1, alpha)

def taylor_series_sqrt_x_plus_1_alpha(x, a):
  sum_elements = 1
  #print("a={}, x={}".format(a,x.data))
  sum_elements += x*a
  sum_elements += 0.5*a*(a-1)*(x**2)
  sum_elements += (1/6)*a*(a-1)*(a-2)*(x**3)

  return sum_elements

def distance(a, b):
  return a - b

#todo remove
#was d=3
def get_p_value(x, max_val, d=2):
  p = (x * approx_to_1_div_x(max_val))**d
  #print("p = {}".format(p))
  return p

#converges only when 0<x<2
#was n=3
def approx_to_1_div_x(x, n=2):
  mul = 1
  for i in range(n):
    mul *= (1+(1-x)**(2**i))
  return mul


def prob_to_be_in_center(x1, x2, x3, x4, c):
  d1 = distance(x1, c)
  d2 = distance(x2, c)
  d3 = distance(x3, c)
  d4 = distance(x4, c)
  #max_val_1 = max_with_lp(d1, d2)
  #max_val_2 = max_with_lp(d3, d4)
  max_val = max_with_lp(d1, d2,d3, d4)
  prob = get_p_value(d1, max_val) * x1
  #prob = get_p_value(d1, max_val)
  prob += x2 * get_p_value(d2, max_val)
  prob += x3 * get_p_value(d3, max_val)
  prob += x4 * get_p_value(d4, max_val)
  #return prob/2
  return prob

def get_ckks_context():
  # parameters
  #poly_mod_degree = 2**12
  poly_mod_degree = 2**13
  #coeff_mod_bit_sizes = [40, 20, 40]
  coeff_mod_bit_sizes = [35, 25, 25, 25, 25, 25, 25, 25, 25, 35]
  # create TenSEALContext
  context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  # scale of ciphertext to use
  #context.global_scale = 2 ** 20
  context.global_scale = 2 ** 25
  # this key is needed for doing dot-product operations
  context.generate_galois_keys()
  return context

def get_other_context():
  # parameters
  poly_mod_degree = 16384
  #poly_mod_degree = 32768
  #coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
  coeff_mod_bit_sizes = [60,35,35,35,35,35,35,35,35,35,60]
  print("sum of coeff_mod_bit_sizes=", sum(coeff_mod_bit_sizes))
  # create TenSEALContext
  ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  ctx_training.global_scale = 2 ** 35
  ctx_training.generate_galois_keys()
  return ctx_training

def get_bfv_context():
  context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
  return context

In [ ]:
x1 = 0.1
x2 = 0.9
x3 = 0.2
x4 = 0.8
c = 0.7

context = get_other_context()
encrypted_x1 = ts.ckks_tensor(context, [x1])
encrypted_x2 = ts.ckks_tensor(context, [x2])
encrypted_x3 = ts.ckks_tensor(context, [x3])
encrypted_x4 = ts.ckks_tensor(context, [x4])

encrypted_c = ts.ckks_tensor(context, [c])

enc_new_center = prob_to_be_in_center(encrypted_x1, encrypted_x2,encrypted_x3,encrypted_x4, encrypted_c)
new_center_dec = enc_new_center.decrypt().tolist()
print("new_center = ", new_center_dec)

for i in range(4):
    new_center_plain = new_center_dec[0]*0.25
    encrypted_c = ts.ckks_tensor(context, [new_center_plain])
    enc_new_center = prob_to_be_in_center(encrypted_x1, encrypted_x2,encrypted_x3,encrypted_x4, encrypted_c)
    new_center_dec = enc_new_center.decrypt().tolist()
    print("new_center = ", new_center_dec)





sum of coeff_mod_bit_sizes= 435
new_center =  [0.19627694395485792]
new_center =  [0.8426215666692896]


In [ ]:
dim = 2
centers = initial_centers
data = plain_data
plain_centers = plain_recalculate_centers(centers, data, dim)

In [ ]:
data_encrypted_vector = ts.ckks_tensor(context, plain_data)

center_encrypted_vector = ts.ckks_tensor(context, initial_centers)

d = 2
# d > 2 causes an error
dim = 2

for i in range(2):
    C_final = recalculate_centers(d, initial_centers, plain_data, dim)
    #C_final = recalculate_centers(d, center_encrypted_vector, data_encrypted_vector, dim)
    initial_centers = C_final
draw_points(C_final, "C_final")




centers_colors = []
C_final = C_final.decrypt()
for x in data_encrypted_vector:
    if(calc_dist(x,C_final[0]) > calc_dist(x,C_final[1])):
        centers_colors.append(0)
    else:
        centers_colors.append(1)
data = np.array([v.decrypt(sk) for v in data_encrypted_vector ])
centers_colors = np.array([v.decrypt(sk) for v in center_encrypted_vector ])
#data = np.append(data,C_final,axis=0)
centers_colors.append(3)
centers_colors.append(5)
print("ours: ")
print(C_final)
print("sklearn: ")
print(run_kmeans(2, data))
print(data.shape)
print_fig(data, centers_colors,"cluster")

In [ ]:
plain_data = np.array([[-10,-1], [-4,-2], [-6,-10], [-9,-8], [9,10], [9,8], [10,9], [10,4]])
draw_points(plain_data, "Original Data")

#initial centers, chosen arbitrarily
initial_centers = np.array([[-1, -5], [2, 6]])
draw_points(initial_centers, "Initial Centers")

In [ ]:
def print_fig(X, centers_colors, plot_name):
    # Create a scatter plot
    fig, axs = plt.subplots(nrows=1, ncols=1)
    axs.scatter(X[:,0], y=X[:,1], c=centers_colors)
    axs.set_title(plot_name)
    fig.show()


def run_kmeans(n_clusters, data):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data)
    return kmeans.cluster_centers_


def recalculate_centers(d, C, X, dim):
    C_new = copy.copy(C)
    for i in range(dim):
        sum_c = 0
        len_c = 0
        for x in X:
            p = calculate_p_value(d, x, C, i)
            sum_c += p * x
            len_c += p
        C_new[i] = sum_c/len_c
    return C_new


# dist without sqrt
def calc_dist(x1, x2):
    sum_p = (x1 - x2)**1
    #print("sum_p in calc_dist={}".format(sum_p.decrypt().tolist()))

    return sum_p



def calculate_p_value(d, x, c, i):
    d_ref = calc_dist(x, c[i])**d
    d0 = calc_dist(x, c[0])**d
    d1 = calc_dist(x, c[1])**d
    res2 = taylor_series(-d0-d1)*taylor_series(d_ref)
    result = res2
    return result


# taylor series of degree n for e^x function
def taylor_series(x):
    summ = 0
    for n in range(3):
      summ += (1/math.factorial(n))*(x**n)
    return summ


def calculate_p_value_not_pow(d, x, c, i, dim):
    return ((calc_dist(x, c[i], dim))**d)/(calc_dist(x, c[0], dim)**d+calc_dist(x, c[1], dim)**d)


def context():
    #context = ts.context(ts.SCHEME_TYPE.CKKS, 8192,-1 ,coeff_mod_bit_sizes=[60, 40, 40, 60])
    # parameters
    poly_mod_degree = 4096
    coeff_mod_bit_sizes = [40, 20, 40]
    # create TenSEALContext
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    # scale of ciphertext to use
    context.global_scale = 2 ** 20
    # this key is needed for doing dot-product operations
    context.generate_galois_keys()
    return context

def draw_points(data, title):
    fig, axs = plt.subplots(nrows=1, ncols=1)
    axs.scatter(data[:,0], y=data[:,1])
    axs.set_title(title)
    axs.set_xlim([-13, 13])
    axs.set_ylim([-13, 13])
    fig.show()